In [23]:
import sys
sys.path.append("../")

In [24]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [25]:
BUY = 1
SELL = -1
NONE = 0


def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE

In [26]:
pair = "EUR_USD"
df_an = pd.read_csv(f"../data/candles/{pair}_H1.csv")
df_m5 = pd.read_csv(f"../data/candles/{pair}_M5.csv")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [27]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [28]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [29]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [30]:
df_an.direction.value_counts()

direction
 0    62428
 1     2643
-1     2643
Name: count, dtype: int64

In [31]:
df_an.head()

,Unnamed: 0,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,...,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,99,2013-01-11 03:00:00+00:00,1525,1.32590,1.32634,1.32530,1.32560,1.32584,1.32628,1.32522,...,1.32639,1.32536,1.32565,0.004679,0.004717,-0.000038,1.312651,-0.000038,NaN,0
1,100,2013-01-11 04:00:00+00:00,1282,1.32560,1.32614,1.32542,1.32596,1.32555,1.32607,1.32536,...,1.32621,1.32548,1.32601,0.004473,0.004668,-0.000195,1.312955,-0.000195,-0.000038,0
2,101,2013-01-11 05:00:00+00:00,1210,1.32594,1.32657,1.32569,1.32625,1.32589,1.32651,1.32561,...,1.32663,1.32575,1.32630,0.004284,0.004591,-0.000307,1.313258,-0.000307,-0.000195,0
3,102,2013-01-11 06:00:00+00:00,2005,1.32627,1.32630,1.32490,1.32516,1.32622,1.32624,1.32484,...,1.32637,1.32495,1.32521,0.004001,0.004473,-0.000472,1.313528,-0.000472,-0.000307,0
4,103,2013-01-11 07:00:00+00:00,3755,1.32518,1.32774,1.32515,1.32724,1.32513,1.32768,1.32510,...,1.32780,1.32518,1.32730,0.003899,0.004358,-0.000459,1.313838,-0.000459,-0.000472,0


In [32]:
def run_pair(pair, time_d=1):
    df_an = pd.read_csv(f"../data/candles/{pair}_H1.csv")
    df_m5 = pd.read_csv(f"../data/candles/{pair}_M5.csv")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = MACD(df_an)
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()

    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)

    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
                'bid_o', 'bid_h', 'bid_l', 'bid_c',
                'ask_o', 'ask_h', 'ask_l', 'ask_c',
                'direction', 'EMA_100']
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)

    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )

    gt.run_test()
    return gt.df_results

In [33]:
res = []
for p in ["AUD_JPY", "EUR_USD", "GBP_JPY", "USD_JPY"]:
    res.append(dict(pair=p, res=run_pair(p)))

In [ ]:
for r in res:
    print(r['pair'], r['res'].result.sum())

In [ ]:
res = []
for p in ["AUD_JPY", "EUR_USD", "GBP_JPY", "USD_JPY"]:
    res.append(dict(pair=p, res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())